In [1]:
!pip install '../'

Processing /Users/colmb/Library/CloudStorage/OneDrive-Graphcore/code/debugging-tool
  Preparing metadata (setup.py) ... done
  Created wheel for nvis: filename=nvis-0.1-py3-none-any.whl size=35894 sha256=a6eb87d9260482258b83aba95e5cd2c11cf5082a44cf5d50618b22f2d12fead0
  Stored in directory: /Users/colmb/Library/Caches/pip/wheels/a5/df/dc/5259f835417f3a97f1653a062131bd69e67749ca4095ea03fc
Successfully built nvis
  Attempting uninstall: nvis
    Found existing installation: nvis 0.1
    Uninstalling nvis-0.1:
      Successfully uninstalled nvis-0.1

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:


# import pandas as pd
from nvis import vis
from nvis.log.common import read_pickle, TensorType
import re
import pandas as pd
df = read_pickle('../test-data/example-logs.pkl')

# vis.interactive(
#     f= vis.scalar_line,
#     width=1000,
#     df=df,
#     tt=TensorType.Activation,
#     layer='output',
#     col_wrap = 3,
#     scalar_metric=['rm2','min_abs']
# )



In [2]:
df

metadata                                     \
                           name                               type   
0                tok_embeddings  torch.nn.modules.sparse.Embedding   
1                       dropout   torch.nn.modules.dropout.Dropout   
2       layers.0.attention_norm                      model.RMSNorm   
3         layers.0.attention.wq     torch.nn.modules.linear.Linear   
4         layers.0.attention.wk     torch.nn.modules.linear.Linear   
...                         ...                                ...   
153148    layers.0.attention.wk     torch.nn.modules.linear.Linear   
153149    layers.0.attention.wq     torch.nn.modules.linear.Linear   
153150  layers.0.attention_norm                      model.RMSNorm   
153151                  dropout   torch.nn.modules.dropout.Dropout   
153152           tok_embeddings  torch.nn.modules.sparse.Embedding   

                          general_stats                                    \
        tensor_type  step          mean       std      mean_abs   max_abs   
0        Activation     0 -4.262986e-05  0.020015  1.599783e-02  0.102079   
1        Activation     0 -4.262986e-05  0.020015  1.599783e-02  0.102079   
2        Activation     0 -1.977458e-03  0.987685  7.899441e-01  5.083467   
3        Activation     0  2.296686e-03  0.315497  2.528414e-01  1.435886   
4        Activation     0 -2.830928e-03  0.314933  2.510965e-01  1.456772   
...             ...   ...           ...       ...           ...       ...   
153148     Gradient  1000 -8.898307e-11  0.000003  1.231984e-06  0.000131   
153149     Gradient  1000 -1.028476e-09  0.000002  7.743534e-07  0.000083   
153150     Gradient  1000  1.421885e-09  0.000004  2.441213e-06  0.000095   
153151     Gradient  1000  9.915611e-07  0.000418  2.780741e-04  0.006545   
153152     Gradient  1000  9.915611e-07  0.000418  2.780741e-04  0.006545   

                                ... exponent_count                          \
             min_abs       rm2  ...             10  11  12  13  14  15  16   
0       1.826123e-08  0.020015  ...              0   0   0   0   0   0   0   
1       1.826123e-08  0.020015  ...              0   0   0   0   0   0   0   
2       8.873100e-07  0.987686  ...              0   0   0   0   0   0   0   
3       6.053597e-08  0.315506  ...              0   0   0   0   0   0   0   
4       6.128103e-07  0.314945  ...              0   0   0   0   0   0   0   
...              ...       ...  ...            ...  ..  ..  ..  ..  ..  ..   
153148  4.236195e-13  0.000003  ...              0   0   0   0   0   0   0   
153149  0.000000e+00  0.000002  ...              0   0   0   0   0   0   0   
153150  3.126388e-12  0.000004  ...              0   0   0   0   0   0   0   
153151  2.128218e-10  0.000418  ...              0   0   0   0   0   0   0   
153152  2.128218e-10  0.000418  ...              0   0   0   0   0   0   0   

                           metadata  
           -inf  inf          dtype  
0           418    0  float8_e4m3fn  
1           418    0  float8_e4m3fn  
2            34    0  float8_e4m3fn  
3            14    0  float8_e4m3fn  
4            12    0  float8_e4m3fn  
...         ...  ...            ...  
153148  1012634    0  float8_e4m3fn  
153149  1018450    0  float8_e4m3fn  
153150   980333    0  float8_e4m3fn  
153151    29706    0  float8_e4m3fn  
153152    29706    0  float8_e4m3fn  

[153153 rows x 48 columns]

In [3]:
vis.interactive(
    f= vis.scalar_line,
    df=df,
    tt='Activation',
    layer='output',
    col_wrap = 3,
    scalar_metric=['rm2','min_abs']
)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1500px'), outputs=({'output_type': 'display_data', 'data…

In [5]:
vis.interactive(
    width=1400,
    f= vis.scalar_global_heatmap,
    df=df,
    tt='Activation',
    scalar_metric='rm2',
    inc=50
)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1400px'), outputs=({'output_type': 'display_data', 'data…

In [4]:
vis.interactive(
    f=vis.exp_hist,
    df=df,
    layer='output',
    tt='Activation',
    step=100,
    col_wrap =3

)

Output(layout=Layout(width='100%'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chil…

Output(layout=Layout(overflow='scroll hidden', width='1500px'), outputs=({'output_type': 'display_data', 'data…

kind <class 'str'>
tt <class 'str'>
step <class 'int'>
layer <class 'list'>
